## Old one

In [9]:
# Import the YOLO model from the Ultralytics library for object detection and tracking
from ultralytics import YOLO
# Import the OpenCV library for video processing
import cv2

# Load the pretrained YOLO model on COCO dataset
vehicle_model = YOLO("vehicle_detection.pt")
lp_model=YOLO("vehicle_detection.pt")
LP_recognise_model=YOLO("recognise.pt")
# Open the video file for reading using OpenCV's VideoCapture object
cap = cv2.VideoCapture("demo(final).mp4")
# Check if the video file is opened successfully
assert cap.isOpened(), "Error reading video file"

# Retrieve the width, height, and frames per second (fps) of the video
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Create a VideoWriter object to save the output video with Object Tracking
video_writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# Loop through the video frames as long as the video is open
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    # Check if the frame is read successfully
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Perform object tracking on the current frame using the YOLO model
    vehicle_detected = vehicle_model.predict(frame,imgsz=640,conf=0.5)

    # Visualize the results on the frame
    annotated_frame=vehicle_detected[0].plot()

    if len(vehicle_detected[0].boxes.cls.tolist()) > 0:
      for box in vehicle_detected[0].boxes:
        cls=int(box.cls[0]) # index  of predicted object
        if cls != 5:
          x1_veh, y1_veh, x2_veh, y2_veh = map(int, box.xyxy[0])
          cropped_vehicle = frame[y1_veh:y2_veh, x1_veh:x2_veh]
          LP_detected = lp_model.predict(
                    cropped_vehicle, imgsz=640, classes=[5], conf=0.5, max_det=1
                )
          if len(LP_detected[0].boxes.cls.tolist()) > 0:
            x1,y1,x2,y2=map(int,LP_detected[0].boxes.xyxy[0])
            cropped_LP=cropped_vehicle[y1:y2, x1:x2]
            # gray_LP = cv2.cvtColor(cropped_LP, cv2.COLOR_BGR2GRAY)
            # rgb_LP = cv2.cvtColor(gray_LP, cv2.COLOR_GRAY2BGR)
            resized_LP = cv2.resize(cropped_LP, (640, 320))
            recognized_text, recognised_img = recognize_license_plate(resized_LP)
            text = recognized_text
            font_scale = 1
            font_thickness = 2
            text_size, baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)

            # Coordinates for the text
            text_x = x1 + x1_veh
            text_y = y1 + y1_veh - 40

            cv2.rectangle(annotated_frame,(text_x, text_y - text_size[1]),(text_x + text_size[0], text_y + baseline),(255, 255, 255),cv2.FILLED,)
            # Draw the text on the rectangle (black text)
            cv2.putText(annotated_frame,text,(text_x, text_y),cv2.FONT_HERSHEY_SIMPLEX,font_scale,(0, 0, 0),font_thickness,)
            # cv2.putText(annotated_frame, recognized_text, (x1+x1_veh, y1+y1_veh - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
      # vehicle_detected=None


    # Write the annotated frame with Object IDs to the output video
    video_writer.write(annotated_frame)

# Release the video capture, video writer, and close any OpenCV windows
cap.release()
video_writer.release()
cv2.destroyAllWindows()



0: 352x640 1 bike, 2 license_plates, 140.8ms
Speed: 13.1ms preprocess, 140.8ms inference, 1053.0ms postprocess per image at shape (1, 3, 352, 640)

0: 640x320 1 license_plate, 41.8ms
Speed: 1.4ms preprocess, 41.8ms inference, 31.6ms postprocess per image at shape (1, 3, 640, 320)

0: 320x640 1 0, 1 2, 2 3s, 1 7, 1 8, 1 ba, 44.6ms
Speed: 1.0ms preprocess, 44.6ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)
recognised text----- ba 7 2 3 3 0 8

0: 352x640 1 bike, 2 license_plates, 24.9ms
Speed: 2.1ms preprocess, 24.9ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 640)

0: 640x320 1 license_plate, 24.0ms
Speed: 2.7ms preprocess, 24.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 320)

0: 320x640 1 0, 1 2, 2 3s, 2 7s, 1 8, 1 ba, 22.9ms
Speed: 1.3ms preprocess, 22.9ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)
recognised text----- ba 7 3 2 3 0 8 7

0: 352x640 1 bike, 2 license_plates, 23.1ms
Speed: 2.7ms preprocess, 23

In [17]:

from collections import defaultdict
import cv2
from ultralytics.utils.checks import check_imshow, check_requirements
from ultralytics.utils.plotting import Annotator, colors
from shapely.geometry import LineString, Point, Polygon

check_requirements("shapely>=2.0.0")


class ObjectCounter:
    """A class to manage the counting of objects in a real-time video stream based on their tracks."""

    def __init__(
        self,
        names,
        reg_pts=None,
        line_thickness=2,
        view_img=False,
        view_in_counts=True,
        view_out_counts=True,
        draw_tracks=False,
    ):
        """
        Initializes the ObjectCounter with various tracking and counting parameters.

        Args:
            names (dict): Dictionary of class names.
            reg_pts (list): List of points defining the counting region.
            line_thickness (int): Line thickness for bounding boxes.
            view_img (bool): Flag to control whether to display the video stream.
            view_in_counts (bool): Flag to control whether to display the in counts on the video stream.
            view_out_counts (bool): Flag to control whether to display the out counts on the video stream.
            draw_tracks (bool): Flag to control whether to draw the object tracks.
        """
        # Mouse events
        self.is_drawing = False
        self.selected_point = None

        # Region & Line Information
        self.reg_pts = [(20, 400), (1260, 400)] if reg_pts is None else reg_pts
        self.counting_region = None

        # Image and annotation Information
        self.im0 = None
        self.tf = line_thickness
        self.view_img = view_img
        self.view_in_counts = view_in_counts
        self.view_out_counts = view_out_counts

        self.names = names  # Classes names
        self.window_name = "Ultralytics YOLOv8 Object Counter"

        # Object counting Information
        self.in_counts = 0
        self.out_counts = 0
        self.count_ids = []
        self.class_wise_count = {}

        # Tracks info
        self.track_history = defaultdict(list)
        self.draw_tracks = draw_tracks

        # Check if environment supports imshow
        self.env_check = check_imshow(warn=True)

        # Initialize counting region
        if len(self.reg_pts) == 2:
            print("Line Counter Initiated.")
            self.counting_region = LineString(self.reg_pts)
        elif len(self.reg_pts) >= 3:
            print("Polygon Counter Initiated.")
            self.counting_region = Polygon(self.reg_pts)
        else:
            print("Invalid Region points provided, region_points must be 2 for lines or >= 3 for polygons.")
            print("Using Line Counter Now")
            self.counting_region = LineString(self.reg_pts)

        # Define the counting line segment
        self.counting_line_segment = LineString(
            [
                (self.reg_pts[0][0], self.reg_pts[0][1]),
                (self.reg_pts[1][0], self.reg_pts[1][1]),
            ]
        )

    def mouse_event_for_region(self, event, x, y, flags, params):
        """
        Handles mouse events for defining and moving the counting region in a real-time video stream.

        Args:
            event (int): The type of mouse event (e.g., cv2.EVENT_MOUSEMOVE, cv2.EVENT_LBUTTONDOWN, etc.).
            x (int): The x-coordinate of the mouse pointer.
            y (int): The y-coordinate of the mouse pointer.
            flags (int): Any associated event flags (e.g., cv2.EVENT_FLAG_CTRLKEY,  cv2.EVENT_FLAG_SHIFTKEY, etc.).
            params (dict): Additional parameters for the function.
        """
        if event == cv2.EVENT_LBUTTONDOWN:
            for i, point in enumerate(self.reg_pts):
                if (
                    isinstance(point, (tuple, list))
                    and len(point) >= 2
                    and (abs(x - point[0]) < 10 and abs(y - point[1]) < 10)
                ):
                    self.selected_point = i
                    self.is_drawing = True
                    break

        elif event == cv2.EVENT_MOUSEMOVE:
            if self.is_drawing and self.selected_point is not None:
                self.reg_pts[self.selected_point] = (x, y)
                self.counting_region = Polygon(self.reg_pts)

        elif event == cv2.EVENT_LBUTTONUP:
            self.is_drawing = False
            self.selected_point = None

    def get_side_of_line(self, point, line_points):
        """Returns whether a point is on the left or right side of the line.
        point: (x, y) tuple representing the vehicle's position.
        line_points: [(x1, y1), (x2, y2)] representing the two endpoints of the line.
        Returns: A positive value if the point is on one side, negative if on the other.
        """
        x, y = point
        x1, y1 = line_points[0]
        x2, y2 = line_points[1]
        return (x - x1) * (y2 - y1) - (y - y1) * (x2 - x1)

    def extract_and_process_tracks(self, tracks):
        """Extracts and processes tracks for object counting in a video stream."""
        # Annotator Init and region drawing
        annotator = Annotator(self.im0, self.tf, self.names)

        # Draw region or line
        annotator.draw_region(reg_pts=self.reg_pts, color=(104, 0, 123), thickness=self.tf * 2)

        # Extract tracks for OBB or object detection
        track_data = tracks[0].obb or tracks[0].boxes

        if track_data and track_data.id is not None:
            boxes = track_data.xyxy.cpu()
            clss = track_data.cls.cpu().tolist()
            track_ids = track_data.id.int().cpu().tolist()

            # Extract tracks
            for box, track_id, cls in zip(boxes, track_ids, clss):
                # Draw bounding box
                annotator.box_label(box, label=self.names[cls], color=colors(int(track_id), True))

                # Store class info
                if self.names[cls] not in self.class_wise_count:
                    self.class_wise_count[self.names[cls]] = {"IN": 0, "OUT": 0}

                # Draw Tracks
                track_line = self.track_history[track_id]
                track_line.append((float((box[0] + box[2]) / 2), float((box[1] + box[3]) / 2)))
                if len(track_line) > 30:
                    track_line.pop(0)

                # Draw track trails
                if self.draw_tracks:
                    annotator.draw_centroid_and_tracks(
                        track_line,
                        color=colors(int(track_id), True),
                        track_thickness=self.tf,
                    )

                prev_position = self.track_history[track_id][-2] if len(self.track_history[track_id]) > 1 else None

                # Count objects in any polygon
                if len(self.reg_pts) >= 3:
                    is_inside = self.counting_region.contains(Point(track_line[-1]))

                    if prev_position is not None and is_inside and track_id not in self.count_ids:
                        self.count_ids.append(track_id)

                        if (box[0] - prev_position[0]) * (self.counting_region.centroid.x - prev_position[0]) > 0:
                            self.in_counts += 1
                            self.class_wise_count[self.names[cls]]["IN"] += 1
                        else:
                            self.out_counts += 1
                            self.class_wise_count[self.names[cls]]["OUT"] += 1

                # Count objects using line
                elif len(self.reg_pts) == 2:
                    if (
                        prev_position is not None
                        and track_id not in self.count_ids
                        and LineString([(prev_position[0], prev_position[1]), (box[0], box[1])]).intersects(
                            self.counting_line_segment
                        )
                    ):
                        self.count_ids.append(track_id)

                        # Determine the side of movement (IN or OUT)
                        prev_side = self.get_side_of_line(prev_position, self.reg_pts)
                        curr_side = self.get_side_of_line((box[0], box[1]), self.reg_pts)

                        if prev_side * curr_side < 0:  # If the signs are different, it means the vehicle crossed the line
                            if curr_side < 0:  # Right side (considered "IN")
                                self.in_counts += 1
                                self.class_wise_count[self.names[cls]]["IN"] += 1
                            else:  # Left side (considered "OUT")
                                self.out_counts += 1
                                self.class_wise_count[self.names[cls]]["OUT"] += 1

        labels_dict = {}

        for key, value in self.class_wise_count.items():
            if value["IN"] != 0 or value["OUT"] != 0:
                if not self.view_in_counts and not self.view_out_counts:
                    continue
                elif not self.view_in_counts:
                    labels_dict[str.capitalize(key)] = f"OUT {value['OUT']}"
                elif not self.view_out_counts:
                    labels_dict[str.capitalize(key)] = f"IN {value['IN']}"
                else:
                    labels_dict[str.capitalize(key)] = f"IN {value['IN']} OUT {value['OUT']}"

        if labels_dict:
            annotator.display_analytics(self.im0, labels_dict, (104, 31, 17), (255, 255, 255), 10)

    def display_frames(self):
        """Displays the current frame with annotations and regions in a window."""
        if self.env_check:
            cv2.namedWindow(self.window_name)
            if len(self.reg_pts) == 4:  # only add mouse event If user drawn region
                cv2.setMouseCallback(self.window_name, self.mouse_event_for_region, {"region_points": self.reg_pts})
            cv2.imshow(self.window_name, self.im0)
            # Break Window
            if cv2.waitKey(1) & 0xFF == ord("q"):
                return

    def start_counting(self, im0, tracks):
        """
        Main function to start the object counting process.

        Args:
            im0 (ndarray): Current frame from the video stream.
            tracks (list): List of tracks obtained from the object tracking process.
        """
        self.im0 = im0  # store image
        self.extract_and_process_tracks(tracks)  # draw region even if no objects

        if self.view_img:
            self.display_frames()
        return self.im0


if __name__ == "__main__":
    classes_names = {0: "person", 1: "car"}  # example class names
    ObjectCounter(classes_names)


Line Counter Initiated.


In [16]:
def recognize_license_plate(cropped_LP):
    detections = LP_recognise_model.predict(cropped_LP, imgsz=640)

    if len(detections[0].boxes.cls.tolist()) > 0:
        boxes = []
        labels = []
        # Iterate through each detection
        for detection in detections:
            # Check if the detection has the 'boxes' attribute
            # Extract box coordinates and class indices
            boxes += detection.boxes.xyxy.tolist()  # Convert tensor to list
            class_indices = detection.boxes.cls.tolist()

            # Convert class indices to actual labels using the names dictionary
            labels += [detection.names[int(cls_idx)] for cls_idx in class_indices]

        # Combine the bounding boxes with their respective labels
        detections_with_labels = list(zip(boxes, labels))

        # Function to determine if two boxes are in the same row (y-coordinate proximity)
        def are_in_same_row(box1, box2, threshold=25): #25
            return abs(box1[1] - box2[1]) < threshold  # Compare y1 values

        # Sort detections by their y-coordinate first (top to bottom)
        detections_sorted_y = sorted(detections_with_labels, key=lambda x: x[0][1])

        current_row = (
            [detections_sorted_y[0]] if detections_sorted_y else []
        )  # Start with the first detection
        rows = []  # To store rows

        # Group detections into rows based on y1 proximity
        for i in range(1, len(detections_sorted_y)):
            current_box = detections_sorted_y[i][0]
            previous_box = current_row[-1][0]

            if are_in_same_row(current_box, previous_box):
                # If the current detection is in the same row, add it to the current row
                current_row.append(detections_sorted_y[i])
            else:
                # Otherwise, start a new row
                rows.append(current_row)
                current_row = [detections_sorted_y[i]]

        # Don't forget to add the last row
        rows.append(current_row)

        # Sort each row by x-coordinate (left to right)
        rows_sorted = [sorted(row, key=lambda x: x[0][0]) for row in rows]

        # Concatenate the labels row by row
        recognized_text = " ".join(
            [detection[1] for row in rows_sorted for detection in row]
        )
        print("recognised text-----",recognized_text)
        return recognized_text, detections[0].plot()
    else:
        return None, None



In [12]:
# Import necessary libraries
from ultralytics import YOLO
import cv2

# Load the pretrained YOLO model on COCO dataset
vehicle_model = YOLO("vehicle_detection.pt")
lp_model=YOLO("vehicle_detection.pt")
LP_recognise_model=YOLO("recognise.pt")

counting_model = YOLO("vehicle_detection.pt")

# Open video file for reading
#cap = cv2.VideoCapture("demo(final).mp4")

cap = cv2.VideoCapture("demo(final).mp4")
#cap = cv2.VideoCapture("video(1080).mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Set line coordinates for vehicle counting
line_points = [(81, 378), (1885, 378)]

# Initialize the dictionary to hold the classes for counting, excluding license plate
class_names = counting_model.names
classes_count = {key: value for key, value in class_names.items() if value != "license_plate"}
classes_to_count = list(classes_count.keys())

# Video writer for output video
video_writer = cv2.VideoWriter("combined_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize Object Counter
counter = ObjectCounter(
    view_img=True,
    reg_pts=line_points,
    names=counting_model.names,
    draw_tracks=True,
    line_thickness=2,
)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Perform vehicle detection
    vehicle_detected = vehicle_model.predict(frame, imgsz=640, conf=0.5)

    # Annotate detected vehicles on the frame
    annotated_frame = vehicle_detected[0].plot()

    # Process each detected vehicle
    for box in vehicle_detected[0].boxes:
        cls = int(box.cls[0]) # index of the predicted object
        if cls != 5:  # Skip license plates
            x1_veh, y1_veh, x2_veh, y2_veh = map(int, box.xyxy[0])
            cropped_vehicle = frame[y1_veh:y2_veh, x1_veh:x2_veh]
            
            # Detect license plate within the vehicle
            LP_detected = lp_model.predict(cropped_vehicle, imgsz=640, classes=[5], conf=0.5, max_det=1)
            if len(LP_detected[0].boxes.cls.tolist()) > 0:
                x1, y1, x2, y2 = map(int, LP_detected[0].boxes.xyxy[0])
                cropped_LP = cropped_vehicle[y1:y2, x1:x2]
                resized_LP = cv2.resize(cropped_LP, (640, 320))
                
                # License Plate Recognition
                recognized_text, recognised_img = recognize_license_plate(resized_LP)
                text = recognized_text
                font_scale = 1
                font_thickness = 2
                text_size, baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
                
                # Coordinates for the text
                text_x = x1 + x1_veh
                text_y = y1 + y1_veh - 40

                # Draw the text
                cv2.rectangle(annotated_frame, (text_x, text_y - text_size[1]), (text_x + text_size[0], text_y + baseline), (255, 255, 255), cv2.FILLED)
                cv2.putText(annotated_frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), font_thickness)

    # Perform vehicle tracking and counting
    tracks = counting_model.track(frame, persist=True, show=False, classes=classes_to_count)
    annotated_frame = counter.start_counting(annotated_frame, tracks)

    # Write the combined frame with all annotations to the output video
    video_writer.write(annotated_frame)

# Release resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()


Line Counter Initiated.

0: 384x640 1 license_plate, 37.6ms
Speed: 2.0ms preprocess, 37.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 scooter, 38.1ms
Speed: 2.0ms preprocess, 38.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 37.0ms
Speed: 5.5ms preprocess, 37.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 scooter, 36.7ms
Speed: 4.0ms preprocess, 36.7ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 1 scooter, 36.1ms
Speed: 2.0ms preprocess, 36.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x320 1 license_plate, 36.4ms
Speed: 1.0ms preprocess, 36.4ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 320)

0: 320x640 (no detections), 36.1ms
Speed: 0.0ms preprocess, 36.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 384x640 1 scooter, 37.2ms
Speed: 3.5ms pre


0: 640x288 (no detections), 26.6ms
Speed: 1.1ms preprocess, 26.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 288)

0: 384x640 1 scooter, 30.6ms
Speed: 2.0ms preprocess, 30.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 1 scooter, 29.7ms
Speed: 2.0ms preprocess, 29.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x320 1 license_plate, 29.7ms
Speed: 2.0ms preprocess, 29.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 320)

0: 320x640 (no detections), 28.6ms
Speed: 1.0ms preprocess, 28.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 384x640 1 scooter, 31.0ms
Speed: 2.5ms preprocess, 31.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 1 scooter, 29.6ms
Speed: 3.5ms preprocess, 29.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x288 (no detections), 27.7ms
Speed: 1.0ms preprocess


0: 320x640 (no detections), 28.0ms
Speed: 1.0ms preprocess, 28.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 384x640 1 scooter, 31.1ms
Speed: 1.5ms preprocess, 31.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bike, 1 license_plate, 29.3ms
Speed: 2.0ms preprocess, 29.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x288 1 license_plate, 26.6ms
Speed: 1.0ms preprocess, 26.6ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 288)

0: 320x640 (no detections), 28.1ms
Speed: 2.0ms preprocess, 28.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 384x640 1 bike, 29.6ms
Speed: 3.5ms preprocess, 29.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bike, 1 license_plate, 29.6ms
Speed: 3.0ms preprocess, 29.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x288 1 license_plate, 26.1ms
Speed: 2.5ms preprocess, 26.1ms 


0: 384x640 2 scooters, 30.7ms
Speed: 3.0ms preprocess, 30.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 2 scooters, 29.6ms
Speed: 2.0ms preprocess, 29.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x288 1 license_plate, 27.6ms
Speed: 1.0ms preprocess, 27.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 288)

0: 320x640 (no detections), 28.5ms
Speed: 1.0ms preprocess, 28.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 640x288 1 license_plate, 26.2ms
Speed: 1.0ms preprocess, 26.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 288)

0: 320x640 (no detections), 27.5ms
Speed: 2.0ms preprocess, 27.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 384x640 2 scooters, 31.2ms
Speed: 3.0ms preprocess, 31.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 2 scooters, 29.5ms
Speed: 2.0ms prepr


0: 384x640 1 license_plate, 2 scooters, 29.5ms
Speed: 3.0ms preprocess, 29.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 640x288 1 license_plate, 27.5ms
Speed: 2.0ms preprocess, 27.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 288)

0: 320x640 (no detections), 28.0ms
Speed: 2.0ms preprocess, 28.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 640x320 (no detections), 29.4ms
Speed: 2.0ms preprocess, 29.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 320)

0: 384x640 2 scooters, 30.6ms
Speed: 2.0ms preprocess, 30.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 2 scooters, 29.6ms
Speed: 3.0ms preprocess, 29.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x288 1 license_plate, 28.4ms
Speed: 1.0ms preprocess, 28.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 288)

0: 320x640 (no detections), 28.6ms
Speed: 1.3ms p

...


0: 640x640 1 license_plate, 50.0ms
Speed: 10.5ms preprocess, 50.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 37.0ms
Speed: 0.0ms preprocess, 37.0ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 352x640 (no detections), 43.6ms
Speed: 0.0ms preprocess, 43.6ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

0: 544x640 (no detections), 50.7ms
Speed: 2.6ms preprocess, 50.7ms inference, 1.6ms postprocess per image at shape (1, 3, 544, 640)

0: 384x640 1 bike, 2 cars, 36.8ms
Speed: 3.0ms preprocess, 36.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bike, 2 cars, 36.5ms
Speed: 3.5ms preprocess, 36.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 license_plate, 56.3ms
Speed: 5.0ms preprocess, 56.3ms inference, 4.9ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 33.7ms
Speed: 2.5ms preprocess, 33.7ms in


0: 576x640 (no detections), 49.9ms
Speed: 3.0ms preprocess, 49.9ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 384x640 2 cars, 35.0ms
Speed: 5.0ms preprocess, 35.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 39.6ms
Speed: 0.0ms preprocess, 39.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 license_plate, 49.7ms
Speed: 5.1ms preprocess, 49.7ms inference, 10.2ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 32.8ms
Speed: 6.5ms preprocess, 32.8ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 608x640 (no detections), 60.2ms
Speed: 0.0ms preprocess, 60.2ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)

0: 384x640 2 cars, 45.9ms
Speed: 0.0ms preprocess, 45.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 38.8ms
Speed: 10.2ms preprocess, 38.8ms inference, 4.0ms postprocess per im


0: 640x640 (no detections), 59.7ms
Speed: 0.0ms preprocess, 59.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 43.3ms
Speed: 0.0ms preprocess, 43.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 42.4ms
Speed: 0.0ms preprocess, 42.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 (no detections), 56.0ms
Speed: 0.0ms preprocess, 56.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 car, 40.0ms
Speed: 0.0ms preprocess, 40.0ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 39.8ms
Speed: 0.0ms preprocess, 39.8ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)

0: 608x640 1 license_plate, 60.3ms
Speed: 0.0ms preprocess, 60.3ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)

0: 320x640 (no detections), 30.3ms
Speed: 9.8ms preprocess, 30.3ms inference, 0.0ms postprocess per image at


0: 640x96 (no detections), 18.5ms
Speed: 0.9ms preprocess, 18.5ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 96)

0: 544x640 (no detections), 46.2ms
Speed: 3.1ms preprocess, 46.2ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 384x640 2 cars, 38.8ms
Speed: 0.0ms preprocess, 38.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 29.9ms
Speed: 9.9ms preprocess, 29.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 544x640 1 license_plate, 43.5ms
Speed: 4.5ms preprocess, 43.5ms inference, 11.4ms postprocess per image at shape (1, 3, 544, 640)

0: 320x640 (no detections), 74.0ms
Speed: 1.5ms preprocess, 74.0ms inference, 39.8ms postprocess per image at shape (1, 3, 320, 640)

0: 640x96 (no detections), 44.8ms
Speed: 0.0ms preprocess, 44.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 96)

0: 384x640 2 cars, 30.2ms
Speed: 6.5ms preprocess, 30.2ms inference, 47.6ms postprocess 


0: 384x640 1 car, 1 license_plate, 1 scooter, 40.2ms
Speed: 0.0ms preprocess, 40.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x288 1 license_plate, 37.7ms
Speed: 0.0ms preprocess, 37.7ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 288)

0: 320x640 (no detections), 37.1ms
Speed: 0.0ms preprocess, 37.1ms inference, 3.1ms postprocess per image at shape (1, 3, 320, 640)

0: 384x640 (no detections), 39.4ms
Speed: 0.0ms preprocess, 39.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 scooter, 36.1ms
Speed: 5.0ms preprocess, 36.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 license_plate, 1 scooter, 33.7ms
Speed: 6.3ms preprocess, 33.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 352x640 (no detections), 36.1ms
Speed: 0.0ms preprocess, 36.1ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)

0: 640x288 1 license_plate, 30.